In [23]:
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [24]:
import time
from math import ceil
import yaml, laspy
import lithops
import pandas as pd
import numpy as np
import neilpy
from lithops.storage.cloud_proxy import open, os as os_cl
from lithops import ServerlessExecutor
from lidarpartitioner import lidarutils, utils
from lidarpartitioner.las_partitioner import Partitioner
from lidarpartitioner.lidarutils import *
import lidarpartitioner
from lithops.worker.utils import get_memory_usage
from sklearn.neighbors import KDTree

In [25]:
# datetime object containing current date and time
now = datetime.now()

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")

datet = dt_string.replace(":", "_").replace(" ","_").replace("/","_")

In [26]:
def partition_file(fname, out_dir, num_points, sufix, buffer):
    divider = Partitioner(fname, sufix)
    m_threshold = num_points
    divider.make_partition(out_dir, capacity=m_threshold, buffer=buffer)

In [38]:
def smrf_filter(id, file, smrf_reults, buffer, call_time):
    
    st_map_process = time.time()
    Allprocessing_time = dict()
    Allprocessing_time['Object_name'] = file.split('/')[-1]
    Allprocessing_time['Map_ID'] = id
    Allprocessing_time['Buffer'] = buffer
    Allprocessing_time['Time_Mapstart'] = round(st_map_process - call_time, 3)
    print("ID of the function: %s" % str(id))
    
    
    # read data stream
    st_read = time.time()
    with open(file, 'rb') as f:
        inF = laspy.read(f)
    point_cloud = np.vstack((inF.x, inF.y, inF.z, inF.withheld)).T
    df = pd.DataFrame(data=point_cloud, columns=['X', 'Y', 'Z', 'Withheld'])
    print(df.head(10))
    download_data = time.time() - st_read
    Allprocessing_time['Downloading_Time'] = round(download_data, 3)  
       
    
    # Run SMRF
    st_filter = time.time()
    cellsize = 2
    windows = 12
    slope_threshold = .15 
    elevation_threshold = 0.30
    elevation_scaler = 0.95
    
        
    
    try:
#         Zsmrf, Tsmrf, obj_cells, obj_points = neilpy.smrf(df.X,df.Y,df.Z,df.Withheld,cellsize,windows,slope_threshold,
#                                                       elevation_threshold,elevation_scaler)
        
        Zsmrf, Tsmrf, obj_cells, obj_points = neilpy.smrf(df.X,df.Y,df.Z,cellsize,windows,slope_threshold,
                                                          elevation_threshold,elevation_scaler)
        
        filtering_time = time.time() - st_filter
            
        # Save the ground points 
        st_wres = time.time()
        fname, data = lidarutils.writer_lasfile(inF, obj_points, file, reduce_stream = True)
        en_wres = time.time() - st_wres
    except Exception as e:
        print('Failed!!. Reason: %s' %(e))
        
    Allprocessing_time['Filtering_time'] = round(filtering_time, 3)
    Allprocessing_time['Save_points'] = round(en_wres, 3)
    Allprocessing_time['Maping_time'] = round(time.time() - st_map_process, 3)
    Allprocessing_time['sendResult_time'] = time.time()
    
    return Allprocessing_time, data

def merg_res(results):
    
    st_reducer = time.time()
    pr_time = dict()
    merging = dict()
    Filtering_time, RWData_time, Maping_time, sendResult_time  = [], [], [], []
    Save_points, PeriodTime_Mapstart = [], []
    num_part = len(results)
    buffer = results[0][0]['Buffer']
    print(num_part)
    
    res_mapdata = []
    for res in results:
        obj = dict()
        RWData_time.append(res[0]['Downloading_Time'])
        Filtering_time.append(res[0]['Filtering_time'])
        Save_points.append(res[0]['Save_points'])
        Maping_time.append(res[0]['Maping_time'])
        PeriodTime_Mapstart.append(res[0]['Time_Mapstart'])
        sendResult_time.append(st_reducer - res[0]['sendResult_time'])
        obj['Object_name'] = res[0]['Object_name']
        obj['data_stream'] = res[1]
        res_mapdata.append(obj)
    del results
    
    # Start merging all the results
    st_merg = time.time()
    m_res = merg_files(res_mapdata, num_part, buffer)
    
    merging_time = time.time() - st_merg
    merging['Merging_time'] = merging_time
    
    pr_time['Funcs_numbers'] = num_part
    pr_time['File_name'] = m_res['File_name']
    pr_time['Points_number'] = m_res['Points_number']
    pr_time['Time_Mapstart'] = min(PeriodTime_Mapstart)
    pr_time['Time_Redstart'] = round(min(sendResult_time), 3)
    pr_time['Downloading_Time'] = max(RWData_time)
    pr_time['Filtering_time'] = max(Filtering_time)
    pr_time['Save_points'] = max(Save_points)
    pr_time['Maping_time'] = max(Maping_time)
    pr_time['Saving_data_time'] = m_res['Saving_data_time']
    pr_time['Merging_files_time'] = m_res['Merging_files_time']
    pr_time['Merged_uploading_time'] = m_res['Merged_uploading_time']
    pr_time['Merging_process'] = m_res['Merging_process']
    pr_time['Reduction_time'] = round(time.time() - st_reducer, 3)
    
    return pr_time

In [39]:
%%time
processing_time = []
data_folder = 'data'
chip_folder = 'chipres'
res_map_folder = 'smrf_reults'
res_red_folder = 'merged_files'
fname = data_folder + '/' + "lasfile.las"
#"Coloreado (RGB) 2016 - PNOA-2016-CAT-352-4556-ORT-CLA-COL.las" "lasfile.las"
# "Coloreado (RGB) 2016 - PNOA-2016-CAT-352-4554-ORT-CLA-COL.las"
# "Coloreado (RGB) 2016 - PNOA-2016-CAT-338-4556-ORT-CLA-COL.las"
folders = [chip_folder, res_map_folder]

for i in range(2):
    
    
    if i > 0:
        
        # Start partitioning
        # Clean folder in bucket if exists, if not create it
        [lidarutils.clean_folder(folder) for folder in folders]
    
        # Set variables and start partitioning
        partitions = 2**i
        buffer = 0
        num_points = ceil(343306/partitions) #5492898 3791954 1373224 343306
        print(num_points)
        
        partition_args = [(fname, chip_folder, num_points, i, buffer) for i in range(1)]
        st_split = time.time()
        with ServerlessExecutor(runtime_memory=3072) as exec: #runtime_memory=3072
            exec.map(partition_file, partition_args)
            exec.get_result()
        end_split = time.time() - st_split
        print(f'Total partition time: {end_split} s')
    
    # Processing files resulting from the partition stage 
    st_processing = time.time()
    if i == 0:
        map_runtime_memory = 2048
    elif i > 4:
        map_runtime_memory = 512
    else:
        map_runtime_memory = 1024
        
    files = os_cl.listdir(chip_folder)
    process_args = [(chip_folder + '/' + file, res_map_folder, buffer, st_processing) for file in files] if i > 0 \
                    else [(fname, res_map_folder, 0, st_processing)]
    print(process_args)
    with ServerlessExecutor(runtime="ammarokran/new-smrf-conda36:1.0.6") as exec:
        exec.map_reduce(smrf_filter, process_args, merg_res, map_runtime_memory=map_runtime_memory,
                        include_modules=['lidarutils', 'lidarpartitioner', 'utils'])
        st_get = time.time()
        res = exec.get_result()
        end_get = time.time() - st_get
        # fexec.clean()
    end_processing = time.time() - st_processing
    res['GettingResult_time'] = round(end_get, 3)
    res['Processing_time'] = round(end_processing, 3)
    processing_time.append(res)
    print(f'Total processing time: {end_processing} s')

2021-09-29 12:58:51,742 [INFO] lithops.config -- Lithops v2.5.0
2021-09-29 12:58:51,759 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: us-east
2021-09-29 12:58:51,759 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-east - Namespace: fadi.hassan@urv.cat_fadi
2021-09-29 12:58:51,759 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID M000 - Selected Runtime: ammarokran/new-smrf-conda36:1.0.6 - 2048MB


[('data/lasfile.las', 'smrf_reults', 0, 1632913131.6278818)]


2021-09-29 12:58:52,735 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID M000 - Starting function invocation: smrf_filter() - Total: 1 activations
2021-09-29 12:58:52,735 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID M000 - View execution logs at C:\Users\Ammar\AppData\Local\Temp\lithops\logs\6a4602-13-M000.log
2021-09-29 12:58:52,739 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID R000 - Selected Runtime: ammarokran/new-smrf-conda36:1.0.6 - 256MB
2021-09-29 12:58:53,044 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID R000 - Starting function invocation: merg_res() - Total: 1 activations
2021-09-29 12:58:53,052 [INFO] lithops.invokers -- ExecutorID 6a4602-13 | JobID R000 - View execution logs at C:\Users\Ammar\AppData\Local\Temp\lithops\logs\6a4602-13-R000.log
2021-09-29 12:58:53,056 [INFO] lithops.wait -- ExecutorID 6a4602-13 - Getting results from functions


    0%|          | 0/2  

2021-09-29 12:59:10,425 [INFO] lithops.executors -- ExecutorID 6a4602-13 - Cleaning temporary data


...

2021-09-29 12:59:10,782 [INFO] lithops.config -- Lithops v2.5.0
2021-09-29 12:59:10,814 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: us-east
2021-09-29 12:59:10,816 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: us-east - Namespace: fadi.hassan@urv.cat_fadi
2021-09-29 12:59:10,820 [INFO] lithops.invokers -- ExecutorID 6a4602-14 | JobID M000 - Selected Runtime: lithopscloud/ibmcf-python-v36 - 3072MB


smrf_reults folder is not exist!
171653


2021-09-29 13:00:21,129 [INFO] lithops.invokers -- ExecutorID 6a4602-14 | JobID M000 - Starting function invocation: partition_file() - Total: 1 activations
2021-09-29 13:00:21,133 [INFO] lithops.invokers -- ExecutorID 6a4602-14 | JobID M000 - View execution logs at C:\Users\Ammar\AppData\Local\Temp\lithops\logs\6a4602-14-M000.log
2021-09-29 13:00:21,137 [INFO] lithops.wait -- ExecutorID 6a4602-14 - Getting results from functions


    0%|          | 0/1  

TypeError: too many positional arguments

In [ ]:
# SpeedUp 
par_times = []
df = pd.DataFrame(processing_time)
seq_time = df.iloc[0]['Processing_time']

par_times = [df.iloc[i]['Processing_time'] for i in range(1, len(df))]
SpUp = [str(round((seq_time/v), 5))+'x' for v in par_times]
SpUp.insert(0, str(0) + ' (base)')
# # print(seq_time, diff)
print('SpeedUp %s' % (SpUp))
df_spup = pd.DataFrame(SpUp, columns=['SpeedUp'])
# df_spup

In [ ]:
df_spup['Funcs_num'] = df['Funcs_numbers']
df_spup['Processing_time'] = df['Processing_time']
df_spup = df_spup[['Funcs_num', 'Processing_time', 'SpeedUp']]

In [ ]:
df_spup

In [ ]:
df2 = pd.DataFrame(processing_time)
A = list(df2['Processing_time'])

ind = np.arange(len(A))    # the x locations for the groups
width = 0.7       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, A, edgecolor='white', width=width)

plt.grid(axis='y', alpha=0.5, ls='--', zorder=0)
plt.xlabel('Number of Functions')
plt.ylabel('Time by Second (s)')
# plt.title('The processing time by using Lithops framework')
plt.xticks(ind, list(df2['Funcs_numbers']))
plt.savefig('./figures_outlier/Processing Time.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df2['Proc_time'] = df2['Filtering_time'] + df2['Save_points']

In [ ]:
A = list(df2['Downloading_Time'])
B = list(df2['Proc_time'])
C = list(df2['Time_Redstart'])
D = list(df2['Reduction_time'])
# E = list(df2['Reduction_time'])

bar1 = np.add(A, B).tolist()
bar2 = np.add(A, np.add(B, C)).tolist()
# bar3 = np.add(np.add(A, B), np.add(C, D)).tolist()
ind = np.arange(len(B)) 
width = 0.7

p1 = plt.bar(ind, A, edgecolor='white', width=width, color='purple')
p2 = plt.bar(ind, B, edgecolor='white', width=width,
             bottom=A)
p3 = plt.bar(ind, C, edgecolor='white', width=width,
             bottom=bar1, color='grey')
p4 = plt.bar(ind, D, edgecolor='white', width=width,
             bottom=bar2)
# p5 = plt.bar(ind, E, edgecolor='white', width=width,
#              bottom=bar3)
plt.grid(axis='y', alpha=0.5, ls='--', zorder=0)
plt.xlabel('Number of Functions')
plt.ylabel('Time by Second (s)')
# plt.title('Approach #1: Time of Processing at a time %s'% (dt_string))
plt.xticks(ind, list(df2['Funcs_numbers']))
plt.legend((p1[0], p2[0], p3[0], p4[0]), ('Downloading Time(C1)', 'Processing Time(C2)', 'Obtaining Results Time(C3)', 
                                   'Reduction Time(C4)'), loc='upper left', bbox_to_anchor=(0.41, 1.02))
plt.savefig('./figures_outlier/Processing Time plot at time %s.png'% (datet), dpi=300, bbox_inches='tight')
plt.show()